## Customer Dataset Cleaning

In [1]:
# %pip install fuzzywuzzy
# !python -m pip install --upgrade pip
# !pip install pycountry
# %pip install python-Levenshtein
# %pip install pandas numpy matplotlib seaborn scikit-learn jupyter

In [2]:
# Activate kernal:
# .\venv_fyp\Scripts\activate

# Step 1: Import libraries
import pandas as pd
import numpy as np
import os
import pycountry
import re
import requests
import time
from datetime import datetime, date
from fuzzywuzzy import process, fuzz

In [3]:
# Step 2: Load dataset
# Replace 'customer_dataset.csv' with your actual file name or path
file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun/2021 - 2025 Customer - Copy.csv"

original_customer_dataset_name = "2021 - 2025 Customer - Copy.csv"

# Read dataset
customer_df = pd.read_csv(file_path)

# Show first few rows (original raw data)
customer_df.head()

COUNTRY = 'Malaysia'

### Data Profiling

In [4]:
## Create later

### Perform Data Cleaning Pipeline - CustomerDataset

In [ ]:
# ============================================= STAGE 1: SCHEMA & COLUMN VALIDATION =============================================
# # Optional columns
def check_optional_columns(df, threshold=0.8):
    """
    Check optional columns for fill percentage and drop columns that are mostly empty.
    Returns the modified DataFrame and a friendly message.
    """
    print("[LOG] Running check_optional_columns...")
    optional_columns = ["date of birth", "gender"]
    dropped_columns = []

    for col in optional_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df) 
            print(f"[LOG] Optional column '{col}' fill ratio: {fill_ratio:.2f}")
            if fill_ratio < threshold:
                dropped_columns.append(col)
                df.drop(columns=[col], inplace=True)  # Drop the column immediately
                # df[col].count(): This counts the number of non-missing (non-null/non-NaN) values in the current column (col).
                # len(df): This gives the total number of rows in the DataFrame.
                # fill_ratio: The division calculates the proportion of filled (non-missing) values in that column. A ratio of 1.0 means the column is entirely filled; a ratio of 0.1 means 90% of the values are missing.
                print(f"[LOG] Dropped optional column '{col}' due to too many missing values")
        else:
            print(f"[LOG] Optional column '{col}' not found")

    # Generate user-friendly message
    if dropped_columns:
        dropped_str = ", ".join(dropped_columns)
        message = (
            f"We noticed that very few entries were provided for {dropped_str}. "
            "These columns have been removed. "
            "Segmentation will still be performed using geographic (City, State) "
            "and behavioral data (e.g., orders, purchase items, total spend)."
        )
    else:
        message = "All optional columns have enough data and are kept for analysis."
    
    return df, message

# Mandatory columns 
def check_mandatory_columns(df, threshold=0.8):
    """
    Check mandatory columns for missing values (column-wise).
    Does not drop columns — only warns user if any column is too incomplete.
    Returns the DataFrame and a message summarizing issues.
    """
    print("[LOG] Running check_mandatory_columns...")
    mandatory_columns = ["customerid", "city", "state"]
    missing_report = []
    warning_columns = []

    for col in mandatory_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df)
            print(f"[LOG] Mandatory column '{col}' fill ratio: {fill_ratio:.2f}")
            missing_percent = (1 - fill_ratio) * 100
            missing_report.append(f"{col}: {missing_percent:.1f}% missing")

            # Warn if missing exceeds threshold
            if fill_ratio < threshold:
                warning_columns.append(col)
        else:
            print(f"[LOG] Mandatory column '{col}' not found")
            # Handle case where column completely missing
            missing_report.append(f"{col}: column not found (100% missing)")
            warning_columns.append(col)

    # Generate friendly message
    if warning_columns:
        warning_str = ", ".join(warning_columns)
        message = (
            f"Some key fields have a high number of missing values: {warning_str}. "
            "The system will still continue cleaning and processing, "
            "but missing values will be handled automatically by our system. "
            "Please ensure your source data is as complete as possible for more accurate segmentation results.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )
    else:
        message = (
            "All mandatory columns have sufficient data and are ready for cleaning.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )

    return df, message

# ============================================= STAGE 2: REMOVE DUPLICATE ENTRY ROW =================================================
def remove_duplicate_entries(df):
    """Remove duplicate rows, keeping the first occurrence"""
    print("[LOG] Running remove_duplicate_entries...")
    initial_len = len(df)
    df = df.drop_duplicates(keep='first').copy()
    print(f"[LOG] Removed {initial_len - len(df)} duplicate rows.")
    return df

# ============================================= STAGE 3: STANDARDIZATION & NORMALIZATION =============================================

def normalize_columns_name(df):
    """Normalize column names: lowercase, strip spaces"""
    print("[LOG] Running normalize_columns_name...")
    df.columns = df.columns.str.strip().str.lower()
    print(f"[LOG] Columns after normalization: {list(df.columns)}")
    return df

# ===============================================================================

def standardize_customer_id(df):
    """Standardize CustomerID format while preserving NaN"""
    print("[LOG] Running standardize_customer_id...")
    if 'customerid' in df.columns:
        # Fill NaN with empty string before converting to string
        df['customerid'] = df['customerid'].fillna('').astype(str).str.strip().str.upper()
        print("[LOG] CustomerID column standardized")
    else:
        print("[LOG] CustomerID column not found, skipping")
    return df
# Might have special case of dirty data exist such as "****", "1234....", "annbwbciwbciowb"
# not sure how to handle it (Currently will say bcs we focus on small business enterprise that have use digital system, so normally customerID will not have inconsistent format issue, even the inconsistant format exist, at the end this row will not be use as when we merge we cant found that customerID)

# ===============================================================================

def standardize_dob(df):
    """Standardize Date of Birth column and convert to YYYY-MM-DD"""
    print("[LOG] Running standardize_dob...")
    # Rename only 'date of birth' to 'dob'
    df = df.rename(columns={'date of birth': 'dob'})  
    if 'dob' in df.columns:
        print("[LOG] DOB column found, parsing dates...")
        def parse_date(x):
            if pd.isnull(x):
                return "Unknown"
            for format in ("%d/%m/%Y", "%m-%d-%y", "%Y-%m-%d", "%d-%b-%Y", "%d-%m-%Y"):    
                try:
                    return datetime.strptime(str(x), format).date() # Final format: YYYY-MM-DD | 2025-10-15
                except Exception:
                    continue
            return "Unknown"  # If no valid format found
        df['dob'] = df['dob'].apply(parse_date)
        print("[LOG] DOB parsing complete")
    else:
        print("[LOG] DOB column not found, skipping")
    return df

# %d/%m/%Y → 12/05/2000
# %m-%d-%y → 05-12-00
# %Y-%m-%d → 2000-05-12
# %d-%b-%Y → 12-May-2000
# %d-%m-%Y → 12-5-2000

# ===============================================================================

def derive_age_features(df):
    """Derive Age from DOB"""
    print("[LOG] Running derive_age_features...")
    if 'dob' in df.columns:
        today = date.today()
        df['age'] = df['dob'].apply(
            lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day))
            if pd.notnull(x) else "Unknown"
        )
        print("[LOG] Age derived from DOB")
    else:
        print("[LOG] DOB column not found, skipping")
    return df
# Example: ((today.month, today.day) < (x.month, x.day))
# (10,15) < (12,1) → True (birthday in Dec is after Oct 15)
# (10,15) < (10,16) → True (birthday tomorrow)
# (10,15) < (5,20) → False (birthday already passed)

# This function calculates each person’s age from their date of birth (dob) by subtracting their birth year from the current year and adjusting if their birthday hasn’t occurred yet this year.

# ===============================================================================

def derive_age_group(df):
    """Derive Age Group based on defined buckets"""
    print("[LOG] Running derive_age_group...")
    if 'age' in df.columns:
        def categorize_age(age):
            if pd.isnull(age):
                return 'Unknown'
            if age < 18: return 'Below 18'
            elif 18 <= age <= 24: return '18-24'
            elif 25 <= age <= 34: return '25-34'
            elif 35 <= age <= 44: return '35-44'
            elif 45 <= age <= 54: return '45-54'
            elif 55 <= age <= 64: return '55-64'
            else: return 'Above 65'
        df['age_group'] = df['age'].apply(categorize_age)
        print("[LOG] Age groups derived")
    else:
        print("[LOG] Age column not found, skipping")
    return df
# ===============================================================================

def drop_dob_after_age_derived(df):
    """Drop DOB column after deriving age and age_group"""
    print("[LOG] Running drop_dob_after_age_derived...")
    if 'dob' in df.columns:
        df = df.drop(columns=['dob'])
        print("[LOG] Dropped DOB column")
    else:
        print("[LOG] DOB column not found, skipping")
    return df

# =================================================================================

def standardize_gender(df):
    """Clean and standardize gender values"""
    print("[LOG] Running standardize_gender...")
    if 'gender' in df.columns:
        # Clean text (remove spaces, make lowercase)
        df['gender'] = df['gender'].astype(str).str.strip().str.lower()

        # Standardize using keyword detection
        def detect_gender(value):
            if any(word in value for word in ['m', 'male', 'man', 'boy']):
                return 'Male'
            elif any(word in value for word in ['f', 'female', 'woman', 'girl']):
                return 'Female'
            else:
                return 'Unknown'

        df['gender'] = df['gender'].apply(detect_gender)
        print("[LOG] Gender standardized")
    else:
        print("[LOG] Gender column not found, skipping")
    return df

# ==================================================================================

def standardize_location(df):
    """Standardize City, and State fields"""
    print("[LOG] Running standardize_location...")
        
    # Helper function: detect suspicious city names
    def is_suspicious_city(name):
        if not name or name.strip() == '':
            return True
        name = str(name).strip()
        
        if len(name) < 2 or len(name) > 50: # Too short or too long
            return True
        
        if re.search(r'[^A-Za-z\s\'-]', name):  # Contains non-alphabetic or weird symbols   # letters, space, apostrophe, dash allowed
            return True
        
        if re.search(r'(.)\1{3,}', name):   # Repeated characters (e.g., "Ccciiiittty")
            return True
        return False

    # --- City ---
    if 'city' in df.columns:
        df['city'] = df['city'].fillna('').astype(str).str.title().str.strip()
        
        # Count entries that are either suspicious or "Others"/"other"
        suspicious_count = df['city'].apply(
            lambda x: is_suspicious_city(x) or x.lower() in ['others', 'other']
        ).sum()
        
        # Replace suspicious or "Others"/"other" entries with "Unknown"
        df['city'] = df['city'].apply(
            lambda x: 'Unknown' if (is_suspicious_city(x) or x.lower() in ['others', 'other']) else x
        )
        
        print(f"[LOG] Standardized 'city'. Suspicious/unknown entries set to 'Unknown': {suspicious_count}")
    else:
        print("[LOG] 'city' column not found, skipping city standardization")
    
    # --- State ---
    if 'state' in df.columns:
        malaysia_states = [sub.name for sub in pycountry.subdivisions if sub.country_code == 'MY']
        df['state'] = df['state'].fillna('').astype(str).str.title().str.strip()
        df['state'] = df['state'].apply(
            lambda x: process.extractOne(x, malaysia_states, scorer=fuzz.token_sort_ratio)[0] if x else 'Unknown'
        )
    else:
        print("[LOG] 'state' column not found, skipping state standardization")

    return df

# ============================================= STAGE 4: MISSING VALUE HANDLING =============================================

def handle_missing_values(df):
    """
    Handle missing values using a column-based approach.
    Each column is treated independently based on its type and business logic.
    """
    print("[LOG] Running handle_missing_values...")
    
    # ---- CONFIGURATION ----
    API_KEY = "68f8ce9a38c3f632237334dyiedb96e"  # Replace with your Maps.co API key
    GEOCODE_URL = "https://geocode.maps.co/search"
    SLEEP_TIME = 1.2  # delay between API calls to avoid rate limit
    
    # ----- 1. Customer ID -----
    if 'customerid' in df.columns:
        before_drop = len(df)
        df = df[df['customerid'].notna()].copy()  # Drop rows without ID
        print(f"[LOG] Dropped {before_drop - len(df)} rows without CustomerID")
    else:
        print("[LOG] 'customerid' column missing, skipping drop")
        
    # ----- 2. Age -----
    if 'age' in df.columns:
        missing_ratio = df['age'].isna().mean()
        print(f"[LOG] Age missing ratio: {missing_ratio:.2%}")
        if missing_ratio > 0:
            if 'gender' in df.columns and df['gender'].nunique() > 1:
                # Group by gender if available
                df['age'] = df.groupby('gender')['age'].transform(
                    lambda x: x.fillna(x.median())
                )
                print("[LOG] Applied gender-based median imputation for missing age")
                
                # Always fill any remaining missing values with overall median
                df['age'] = df['age'].fillna(df['age'].median())
                
                # Update derived column if needed
                df = derive_age_group(df)
        else:
            print("[LOG] No missing values for age")
    else:
        print("[LOG] 'age' column missing, skipping age imputation")

    # ----- 4. Gender -----
    if 'gender' in df.columns:
        gender_mode = df['gender'].mode()[0]    # 1. Calculate the mode (most frequent value)
        unknown_count = (df['gender'] == 'Unknown').sum()
        df['gender'] = df['gender'].replace('Unknown', gender_mode)  # 2. Replace 'Unknown' values with the calculated mode
        print(f"[LOG] Replaced {unknown_count} 'Unknown' gender values with mode: {gender_mode}")
    else:
        print("[LOG] 'gender' column missing, skipping gender imputation")
        
    # ----- 5. City & State-----
    if {'city', 'state'}.issubset(df.columns):
        # --- Case 1: city present, state missing ---
        print("\n🔍 [Case 1] Filling missing state using Geocoding API (city known)...")
        # Filter only rows needing API call
        rows_to_fill = df[(df['city'] != "Unknown") & (df['state'] == "Unknown")]
        print(f"[LOG] Filling missing states where city known: {len(rows_to_fill)} rows")
        # Optional: cache repeated city lookups
        cache = {}
        for idx, row in rows_to_fill.iterrows():
            city = row['city']

            # Check if we've already queried this city
            if city in cache:
                state_guess = cache[city]
            else:
                try:
                    params = {"q": f"{row['city']}, Malaysia", "api_key": API_KEY}
                    response = requests.get(GEOCODE_URL, params=params, timeout=10)
                    if response.status_code == 200:
                        data = response.json()
                        if isinstance(data, list) and len(data) > 0:
                            address = data[0].get("display_name", "")
                            parts = [p.strip() for p in address.split(",")]
                            if len(parts) >= 3:
                                state_guess = parts[-3]
                                cache[city] = state_guess  # store in cache
                                df.at[idx, 'state'] = state_guess
                                print(f"✅ {row['city']} → Filled state: {state_guess}")
                    time.sleep(SLEEP_TIME)
                except Exception as e:
                    print(f"⚠️ Error for {row['city']}: {e}")

        # --- Case 2: no city but have state ---
        print("\n🔍 [Case 2] Filling missing city using mode per state...")
        mode_city_per_state = df[df['city'] != "Unknown"].groupby('state')['city'].agg(
            lambda x: x.mode().iloc[0] if not x.mode().empty else "Unknown"
        )
        for idx, row in df.iterrows():
            if row['city'] == "Unknown" and row['state'] != "Unknown":
                most_common_city = mode_city_per_state.get(row['state'], "Unknown")
                df.at[idx, 'city'] = most_common_city
                print(f"✅ {row['state']} → Filled city: {most_common_city}")

        # --- Case 3: both city and state unknown ---
        print("\n🔍 [Case 3] Filling rows where both city & state are missing using mode pair...")
        valid_pairs = df[(df['city'] != "Unknown") & (df['state'] != "Unknown")]
        if not valid_pairs.empty:
            mode_pair = valid_pairs.groupby(['city', 'state']).size().idxmax()
            city_mode, state_mode = mode_pair
            mask = (df['city'] == "Unknown") & (df['state'] == "Unknown")
            df.loc[mask, ['city', 'state']] = [city_mode, state_mode]
            print(f"[LOG] Filled {mask.sum()} rows with mode pair → City: {city_mode}, State: {state_mode}")
    else:
        print("[LOG] 'city' or 'state' column missing, skipping location missing value handling")
    
    return df

# ============================================= STAGE 5: OUTLIER DETECTION =============================================

def detect_outliers(df):
    """Adaptive outlier handling based on dataset size."""
    print("[LOG] Running detect_outliers...")
    if 'age' in df.columns:
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
        # df['age_original'] = df['age']  # ✅ Keep a copy of original age values (for comparison or re-deriving age_group)

        n = len(df)
        print(f"[LOG] Dataset has {n} rows")
        if n < 500:
            # IQR method
            Q1 = df['age'].quantile(0.25)
            Q3 = df['age'].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = df[(df['age'] <  Q1) | (df['age'] > Q3)].shape[0]
            df.loc[(df['age'] < lower_bound) | (df['age'] > upper_bound), 'age'] = np.nan
            print(f"[LOG] IQR Applied for {n} rows. Range: [{lower_bound:.1f}, {upper_bound:.1f}] Outliers set to NaN: {outliers}")
        else:
            # Percentile capping
            lower_bound = df['age'].quantile(0.01)
            upper_bound = df['age'].quantile(0.99)
            df['age'] = df['age'].clip(lower=lower_bound, upper=upper_bound)
            print(f"[LOG] Percentile capping  applied for {n} rows. Capped to [{lower_bound:.1f}, {upper_bound:.1f}]")
    else:
        print("[LOG] 'age' column missing, skipping outlier detection")
    return df

# ============================================= STAGE 6: DEDUPLICATION =============================================
# This function removes duplicate CustomerIDs and keeps the row with the most non-missing data to preserve the most complete customer record.
def deduplicate_customers(df):
    """Handle duplicate customer records intelligently"""
    print("[LOG] Running deduplicate_customers...")
    if 'customerid' not in df.columns:
        print("[LOG] 'customerid' column missing, skipping deduplication")
        return df  # Skip if no customerid column
    
    merged_records = []
    total_duplicates = 0
    
    for cust_id, group in df.groupby('customerid'):
        if len(group) > 1:
            total_duplicates += len(group) - 1
        merged_row = {'customerid': cust_id}
        for col in df.columns:
            if col == 'customerid':
                continue
            
            # Get all unique non-null values for this customer
            values = group[col].dropna().unique()
            
            if len(values) == 0:
                merged_row[col] = pd.NA
            elif len(values) == 1:
                merged_row[col] = values[0]
            else:
                # Multiple conflicting values → choose most frequent (mode)
                merged_row[col] = group[col].mode()[0]
        
        merged_records.append(merged_row)
    print(f"[LOG] Deduplicated {total_duplicates} duplicate customer entries")
    return pd.DataFrame(merged_records)


In [ ]:
def clean_customer_dataset(df, original_customer_dataset_name):
    """
    Main cleaning pipeline for customer dataset.
    Executes all stages in proper order:
    0. Column Normalization
    1. Schema & Column Validation
    2. Duplicate Entry Removal
    3. Standardization & Normalization
    4. Missing Value Handling
    5. Outlier Detection
    6. Deduplication
    Finally, saves the cleaned dataset and returns it.
    """
    print("🚀 Starting data cleaning pipeline...\n")

    # =============================================
    # STAGE 0: NORMALIZE COLUMN NAMES
    # =============================================
    print("========== [STAGE 0 START] Normalize Column Names ==========")
    df = normalize_columns_name(df)
    print("✅ [STAGE 0 COMPLETE] Column names normalized.\n")

    # =============================================
    # STAGE 1: SCHEMA & COLUMN VALIDATION
    # =============================================
    print("========== [STAGE 1 START] Schema & Column Validation ==========")
    df, optional_msg = check_optional_columns(df)
    df, mandatory_msg = check_mandatory_columns(df)
    print(optional_msg)
    print(mandatory_msg)
    print("✅ [STAGE 1 COMPLETE] Schema validation done.\n")

    # =============================================
    # STAGE 2: REMOVE DUPLICATE ENTRY ROWS
    # =============================================
    print("========== [STAGE 2 START] Remove Duplicate Entry Rows ==========")
    df = remove_duplicate_entries(df)
    print("✅ [STAGE 2 COMPLETE] Duplicate entries removed.\n")

    # =============================================
    # STAGE 3: DEDUPLICATION
    # =============================================
    print("========== [STAGE 3 START] Deduplication ==========")
    df = deduplicate_customers(df)
    print("✅ [STAGE 3 COMPLETE] Duplicate CustomerIDs deduplicated.\n")

    # =============================================
    # STAGE 4: STANDARDIZATION & NORMALIZATION
    # =============================================
    print("========== [STAGE 3 START] Standardization & Normalization ==========")
    df = standardize_customer_id(df)
    df = standardize_dob(df)
    df = derive_age_features(df)
    df = derive_age_group(df)
    df = drop_dob_after_age_derived(df)
    df = standardize_gender(df)
    df = standardize_location(df)
    print("✅ [STAGE 3 COMPLETE] Standardization and normalization finished.\n")

    # =============================================
    # STAGE 5: MISSING VALUE HANDLING
    # =============================================
    print("========== [STAGE 4 START] Missing Value Handling ==========")
    df = handle_missing_values(df)
    print("✅ [STAGE 4 COMPLETE] Missing values handled.\n")

    # # =============================================
    # # STAGE 6: OUTLIER DETECTION
    # # =============================================
    print("========== [STAGE 5 START] Outlier Detection ==========")
    df = detect_outliers(df)   # make sure detect_outliers returns df
    print("✅ [STAGE 5 COMPLETE] Outliers handled.\n")

    # =============================================
    # SAVE CLEANED DATASET
    # =============================================
    print("========== [FINAL STAGE START] Save Cleaned Dataset ==========")
    base_name, ext = os.path.splitext(original_customer_dataset_name)
    cleaned_file = f"{base_name}_cleaned{ext}"
    df.to_csv(cleaned_file, index=False)
    print(f"✅ [FINAL STAGE COMPLETE] Cleaned dataset saved as: {cleaned_file}\n")

    print("==========================================================")
    print("🎉 Data cleaning pipeline completed successfully!\n")
    return df, cleaned_file

In [7]:
# Keep a copy for comparison
original_customer_df = customer_df.copy()
cleaned_customer_df, cleaned_file_name = clean_customer_dataset(customer_df, original_customer_dataset_name)
cleaned_customer_df.head()
# cleaned_customer_df.info()

🚀 Starting data cleaning pipeline...

========== [STAGE 0 START] Normalize Column Names ==========
[LOG] Running normalize_columns_name...
[LOG] Columns after normalization: ['customerid', 'date of birth', 'gender', 'city', 'state']
✅ [STAGE 0 COMPLETE] Column names normalized.

========== [STAGE 1 START] Schema & Column Validation ==========
[LOG] Running check_optional_columns...
[LOG] Optional column 'date of birth' fill ratio: 0.00
[LOG] Dropped optional column 'date of birth' due to too many missing values
[LOG] Optional column 'gender' fill ratio: 0.00
[LOG] Dropped optional column 'gender' due to too many missing values
[LOG] Running check_mandatory_columns...
[LOG] Mandatory column 'customerid' fill ratio: 1.00
[LOG] Mandatory column 'city' fill ratio: 0.55
[LOG] Mandatory column 'state' fill ratio: 0.53
We noticed that very few entries were provided for date of birth, gender. These columns have been removed. Segmentation will still be performed using geographic (City, State) a

,customerid,city,state
0,CUST0001,Sungai Besi,Wilayah Persekutuan Kuala Lumpur
1,CUST0002,Mutiara Damansara,Selangor
2,CUST0003,Shah Alam,Selangor
3,CUST0004,Muar,Johor
4,CUST0005,Others,Selangor


In [ ]:
cleaned_customer_df.info()
cleaned_customer_df.isna().sum()
#BEFORE and AFTER Report - 24/10/2025

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3199 entries, 0 to 3198
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerid  3199 non-null   object
 1   city        3199 non-null   object
 2   state       3199 non-null   object
dtypes: object(3)
memory usage: 75.1+ KB


customerid    0
city          0
state         0
dtype: int64